In [1]:
import os
import csv

# Directory containing experiment folders
experiments_dir = "experiments"

# Collect rows for each table
parameters_rows = []  # For the parameters table
summary_rows = []  # For the summary table

# Keep track of all parameter keys seen across experiments
all_param_keys = set()

# New summary fields in summary.csv for multiple runs
summary_keys = [
    "Average Best Loss",
    "Std Best Loss",
    "Average Gradient Time",
    "Std Gradient Time",
    "Average Sampling Time",
    "Std Sampling Time",
    "Average PGD Time",
    "Std PGD Time",
    "Average Loss Time",
    "Std Loss Time",
    "Average Total Time",
    "Std Total Time",
]

# We map them to shorter column names and combine avg±std.
summary_column_map = {
    "Average Best Loss": "Loss",
    "Std Best Loss": "Loss",
    "Average Gradient Time": "Grad (s)",
    "Std Gradient Time": "Grad (s)",
    "Average Sampling Time": "Sampling (s)",
    "Std Sampling Time": "Sampling (s)",
    "Average PGD Time": "PGD (s)",
    "Std PGD Time": "PGD (s)",
    "Average Loss Time": "LossTime (s)",
    "Std Loss Time": "LossTime (s)",
    "Average Total Time": "Total (s)",
    "Std Total Time": "Total (s)",
}

# Final column order for the results table (removed Best Iter column)
final_result_columns = [
    "Experiment",
    "Loss",
    "Grad (s)",
    "Sampling (s)",
    "PGD (s)",
    "LossTime (s)",
    "Total (s)",
]


def latex_escape(text: str) -> str:
    """
    Escape underscores so they don't become subscripts in LaTeX.
    Extend as needed for other special characters.
    """
    return text.replace("_", "\\_")


def format_numeric(value: str, param_name: str = "") -> str:
    """
    Attempt to parse and format:
      - If param_name == "seed", interpret as integer (no decimals).
      - Else parse as float and format with 4 decimals.
      - If parsing fails, treat as text and underscore-escape.
    """
    # Special case for booleans
    if value in ("True", "False"):
        return value

    # Special case for integer parameters
    if param_name.lower() in (
        "seed",
        "iter",
        "search_width",
        "min_search_width",
        "num_steps",
    ):
        try:
            return str(int(float(value)))
        except ValueError:
            return latex_escape(value)

    # Try parsing as float with 4 decimals
    try:
        float_val = float(value)
        return f"{float_val:.4f}"
    except ValueError:
        return latex_escape(value)


def combine_avg_std(avg_val: str, std_val: str) -> str:
    """
    Combine avg±std, each with 4-decimal formatting.
    If either avg or std is missing/'nan', show 0.0000±0.0000.
    """
    if (not avg_val) or (avg_val.lower() == "nan") or (not std_val) or (std_val.lower() == "nan"):
        return "0.0000±0.0000"

    try:
        avg_float = float(avg_val)
        std_float = float(std_val)
        return f"{avg_float:.4f}±{std_float:.4f}"
    except ValueError:
        return "0.0000±0.0000"


# Gather the experiment folders in a sorted order
experiment_folders = sorted(
    folder
    for folder in os.listdir(experiments_dir)
    if os.path.isdir(os.path.join(experiments_dir, folder))
)

# Process each experiment folder and assign IDs (exp1, exp2, ...)
for i, folder in enumerate(experiment_folders, start=1):
    exp_id = f"exp{i}"
    folder_path = os.path.join(experiments_dir, folder)

    # ------------------ PARAMETERS TABLE ------------------
    parameters = {}
    params_file = os.path.join(folder_path, "parameters.csv")
    if os.path.exists(params_file):
        with open(params_file, newline="") as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                param = row["Parameter"]
                # Skip the "debug_output" parameter
                if param == "debug_output":
                    continue
                value = row["Value"]
                parameters[param] = value
                all_param_keys.add(param)

    param_row = {"Experiment": exp_id}
    param_row.update(parameters)
    parameters_rows.append(param_row)

    # ------------------ RESULTS TABLE ------------------
    summary = {}
    summary_file = os.path.join(folder_path, "summary.csv")
    if os.path.exists(summary_file):
        # Expecting a CSV file with two columns: Metric,Value
        with open(summary_file, newline="") as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                metric = row["Metric"]
                value = row["Value"]
                summary[metric] = value

    # Build a new row with combined columns
    summary_row = {"Experiment": exp_id}

    # Combine Average Best Loss and Std Best Loss into "Loss"
    summary_row["Loss"] = combine_avg_std(
        summary.get("Average Best Loss", ""), summary.get("Std Best Loss", "")
    )

    # Combine times using the new keys
    summary_row["Grad (s)"] = combine_avg_std(
        summary.get("Average Gradient Time", ""), summary.get("Std Gradient Time", "")
    )
    summary_row["Sampling (s)"] = combine_avg_std(
        summary.get("Average Sampling Time", ""), summary.get("Std Sampling Time", "")
    )
    summary_row["PGD (s)"] = combine_avg_std(
        summary.get("Average PGD Time", ""), summary.get("Std PGD Time", "")
    )
    summary_row["LossTime (s)"] = combine_avg_std(
        summary.get("Average Loss Time", ""), summary.get("Std Loss Time", "")
    )
    summary_row["Total (s)"] = combine_avg_std(
        summary.get("Average Total Time", ""), summary.get("Std Total Time", "")
    )

    summary_rows.append(summary_row)

# ----------------------------------------------------------------------------
# 1) TABLE FOR PARAMETERS
# ----------------------------------------------------------------------------
param_columns = ["Experiment"] + sorted(all_param_keys, key=lambda x: x.lower())

latex_params = (
    "\\begin{table}[ht]\n"
    "\\centering\n"
    "\\resizebox{\\textwidth}{!}{%\n"  # Wrap in \\resizebox
    "  \\begin{tabular}{" + "l" * len(param_columns) + "}\n"
    "  \\hline\n"
)

# Header row (escape underscores in column names)
escaped_headers = [latex_escape(col) for col in param_columns]
latex_params += "  " + " & ".join(escaped_headers) + " \\\\\n"
latex_params += "  \\hline\n"

# Data rows
for row in parameters_rows:
    row_values = []
    for col in param_columns:
        val = row.get(col, "")
        val_formatted = format_numeric(val, param_name=col)
        row_values.append(val_formatted)
    latex_params += "  " + " & ".join(row_values) + " \\\\\n"

latex_params += (
    "  \\hline\n"
    "  \\end{tabular}\n"
    "} % end of resizebox\n"
    "\\caption{Experiment parameters.}\n"
    "\\label{tab:experiment_parameters}\n"
    "\\end{table}\n"
)

# ----------------------------------------------------------------------------
# 2) TABLE FOR RESULTS (SUMMARY)
# ----------------------------------------------------------------------------
latex_summary = (
    "\\begin{table}[ht]\n"
    "\\centering\n"
    "\\resizebox{\\textwidth}{!}{%\n"  # Wrap in \\resizebox
    "  \\begin{tabular}{" + "l" * len(final_result_columns) + "}\n"
    "  \\hline\n"
)

# Header row
latex_summary += "  " + " & ".join(final_result_columns) + " \\\\\n"
latex_summary += "  \\hline\n"

# Data rows
for row in summary_rows:
    row_values = []
    for col in final_result_columns:
        val = row.get(col, "")
        # For combined columns (times and loss), simply escape underscores.
        if col in ["Grad (s)", "Sampling (s)", "PGD (s)", "LossTime (s)", "Total (s)", "Loss"]:
            val_formatted = latex_escape(val)
        else:
            val_formatted = format_numeric(val, param_name=col)
        row_values.append(val_formatted)
    latex_summary += "  " + " & ".join(row_values) + " \\\\\n"

latex_summary += (
    "  \\hline\n"
    "  \\end{tabular}\n"
    "} % end of resizebox\n"
    "\\caption{Summary of experiment results.}\n"
    "\\label{tab:experiment_summary}\n"
    "\\end{table}\n"
)

# Print both tables
print("%% ========== TABLE: PARAMETERS ========== %%")
print(latex_params)
print("\n%% ========== TABLE: SUMMARY ========== %%")
print(latex_summary)


%% ========== TABLE: PARAMETERS ========== %%
\begin{table}[ht]
\centering
\resizebox{\textwidth}{!}{%
  \begin{tabular}{llllllllllll}
  \hline
  Experiment & alpha & dynamic\_search & eps & gcg\_attack & joint\_eval & min\_search\_width & num\_steps & pgd\_after\_gcg & pgd\_attack & search\_width & seed \\
  \hline
  exp1 & 4/255 & False & 64/255 & False & False & 0 & 600 & False & True & 0 & 1 \\
  exp2 & 0/255 & False & 0/255 & True & False & 512 & 250 & False & False & 512 & 1 \\
  exp3 & 4/255 & False & 64/255 & True & False & 0 & 250 & False & True & 512 & 1 \\
  exp4 & 4/255 & False & 64/255 & True & False & 0 & 250 & True & True & 512 & 1 \\
  exp5 & 4/255 & False & 64/255 & True & False & 0 & 600 & False & True & 64 & 1 \\
  exp6 &  &  &  &  &  &  &  &  &  &  &  \\
  exp7 &  &  &  &  &  &  &  &  &  &  &  \\
  \hline
  \end{tabular}
} % end of resizebox
\caption{Experiment parameters.}
\label{tab:experiment_parameters}
\end{table}


%% ========== TABLE: SUMMARY ========== %%
\b